In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv,find_dotenv

_ :bool = load_dotenv("E:/Python/.env")

In [2]:
client : OpenAI = OpenAI()

### Step 1: Create an Assistant

In [3]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Math's Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106",
)


print(assistant)

Assistant(id='asst_xznH4PSflK9vWv6t2MrgcxLn', created_at=1702555241, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-3.5-turbo-1106', name="Math's Tutor", object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])


### Step 2: Create a Thread

In [4]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()


print(thread)

Thread(id='thread_tu4YIWrR6cvtOJe8GRzhFSlz', created_at=1702555245, metadata={}, object='thread')


### Step 3: Add a Message to a Thread

In [5]:
from openai.types.beta.threads.thread_message import ThreadMessage

message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `2x + 5 = 3x`. Can you help me"
)

print(message)

ThreadMessage(id='msg_L0A3ug5lOWv7DYKLMYKrdyR4', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `2x + 5 = 3x`. Can you help me'), type='text')], created_at=1702555248, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_tu4YIWrR6cvtOJe8GRzhFSlz')


### Step 4: Run the Assistant

In [6]:
from  openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions="Please address the user as Asif. The user has a premium account"
)

print(run)

Run(id='run_lj6yWJZpdnYAtEx0j9bNDmN9', assistant_id='asst_xznH4PSflK9vWv6t2MrgcxLn', cancelled_at=None, completed_at=None, created_at=1702555254, expires_at=1702555854, failed_at=None, file_ids=[], instructions='Please address the user as Asif. The user has a premium account', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_tu4YIWrR6cvtOJe8GRzhFSlz', tools=[ToolAssistantToolsCode(type='code_interpreter')])


### Step 5: Check the Run status

In [13]:
import time

def print_run_and_check_status(run):
    current_status = run.status
    while current_status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        current_status = run.status
        print(current_status)

    if run.status == "completed":
        print(run)


In [14]:
print_run_and_check_status(run)

completed
Run(id='run_lj6yWJZpdnYAtEx0j9bNDmN9', assistant_id='asst_xznH4PSflK9vWv6t2MrgcxLn', cancelled_at=None, completed_at=1702555264, created_at=1702555254, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Asif. The user has a premium account', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702555255, status='completed', thread_id='thread_tu4YIWrR6cvtOJe8GRzhFSlz', tools=[ToolAssistantToolsCode(type='code_interpreter')])


### Step 6: Display the Assistant's Response

In [15]:
from typing import List

messages: List[ThreadMessage] = client.beta.threads.messages.list(
    thread_id= thread.id
) 

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)
    

user: I need to solve the equation `2x + 5 = 3x`. Can you help me
assistant: The solution to the equation 2x + 5 = 3x is x = 5.


### Adding another message 

In [16]:
message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="can you teach me, the steps involve in solving the above equation"
)

In [17]:
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions="Please address the user as Asif. The user has a premium account"
)

# Call the function to print and check status changes
print_run_and_check_status(run)

in_progress
in_progress
completed
Run(id='run_9uXHKOK5iqvPcMJezoSAdSDx', assistant_id='asst_xznH4PSflK9vWv6t2MrgcxLn', cancelled_at=None, completed_at=1702555557, created_at=1702555554, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Asif. The user has a premium account', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702555554, status='completed', thread_id='thread_tu4YIWrR6cvtOJe8GRzhFSlz', tools=[ToolAssistantToolsCode(type='code_interpreter')])


In [18]:
from typing import List

messages: List[ThreadMessage] = client.beta.threads.messages.list(
    thread_id= thread.id
) 

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

user: I need to solve the equation `2x + 5 = 3x`. Can you help me
assistant: The solution to the equation 2x + 5 = 3x is x = 5.
user: can you teach me, the steps involve in solving the above equation
assistant: Sure! The steps involved in solving the equation 2x + 5 = 3x are as follows:

Step 1: Start with the given equation 2x + 5 = 3x.

Step 2: Subtract 2x from both sides of the equation to isolate the variable x on one side. This will give us 5 = x.

Step 3: The result is the solution to the equation: x = 5.

So, the solution to the equation is x = 5.


In [19]:
message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what type of these equation are?"
)
run: Run = client.beta.threads.runs.create(
    thread_id= thread.id,
    assistant_id= assistant.id,
    instructions="Please address the user as Asif. The user has a premium account"
)
print_run_and_check_status(run)


in_progress
in_progress
completed
Run(id='run_iwYjTuOl3pJ97QpY69mCm2ca', assistant_id='asst_xznH4PSflK9vWv6t2MrgcxLn', cancelled_at=None, completed_at=1702555601, created_at=1702555597, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Asif. The user has a premium account', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702555598, status='completed', thread_id='thread_tu4YIWrR6cvtOJe8GRzhFSlz', tools=[ToolAssistantToolsCode(type='code_interpreter')])


In [20]:
from typing import List

messages: List[ThreadMessage] = client.beta.threads.messages.list(
    thread_id= thread.id
) 

for m in reversed(messages.data):
    print(m.role + ": " + m.content[0].text.value)

user: I need to solve the equation `2x + 5 = 3x`. Can you help me
assistant: The solution to the equation 2x + 5 = 3x is x = 5.
user: can you teach me, the steps involve in solving the above equation
assistant: Sure! The steps involved in solving the equation 2x + 5 = 3x are as follows:

Step 1: Start with the given equation 2x + 5 = 3x.

Step 2: Subtract 2x from both sides of the equation to isolate the variable x on one side. This will give us 5 = x.

Step 3: The result is the solution to the equation: x = 5.

So, the solution to the equation is x = 5.
user: what type of these equation are?
assistant: The equation 2x + 5 = 3x is a linear equation. In general, a linear equation is an algebraic equation in which each term is either a constant or the product of a constant and (the first power of) a single variable. The standard form of a linear equation is Ax + By = C, where A, B, and C are constants and x and y are variables.
